# This is an implementation for computing Collision Free Regions in Configuration Space using IRIS

[IRIS in Configuration Space](https://drake.mit.edu/doxygen_cxx/group__geometry__optimization.html#ga3a51e0fec449a0abcf498f78a2a390a8)


This notebook contains code for creating, visualizing and saving such regions.

In [1]:
import multiprocessing as mp
import os.path
import sys
import time
from collections import OrderedDict
from typing import Dict

import numpy as np
import pydot
from IPython.display import SVG, display

import random
import time


from pydrake.common.value import AbstractValue
from pydrake.geometry import (
    Meshcat,
    MeshcatVisualizer,
    QueryObject,
    Rgba,
    Role,
    Sphere,
    StartMeshcat,
)
from pydrake.geometry.optimization import (
    HPolyhedron,
    IrisInConfigurationSpace,
    IrisOptions,
    LoadIrisRegionsYamlFile,
    SaveIrisRegionsYamlFile,
)

from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix
from pydrake.multibody.inverse_kinematics import InverseKinematics
from pydrake.multibody.meshcat import JointSliders
from pydrake.multibody.parsing import PackageMap, Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph, MultibodyPlant
from pydrake.multibody.tree import Body
from pydrake.solvers import MathematicalProgram, Solve
from pydrake.systems.framework import DiagramBuilder, LeafSystem
from pydrake.visualization import AddDefaultVisualization, ModelVisualizer

from pydrake.all import(
    MeshcatVisualizer,
    MeshcatVisualizerParams,
)

sys.path.append("/home/github/autonomous-manipulation/")
from src.manipulation import running_as_notebook
from src.manipulation.utils import FindDataResource

from src.auxiliar.importing_objects import (
    add_bin, add_custom_robot, add_custom_wsg, add_objects, 
    add_table, add_shelf, add_table, weld_gripper_to_robot)


# Update the import
from src.auxiliar.auxiliar_functions import ScaleHPolyhedron
from src.auxiliar.auxiliar_functions import _CheckNonEmpty



In [ ]:
# Start the visualizer (only run this once).
meshcat = StartMeshcat()

# Setup



In [ ]:
iris_filename = "/home/github/autonomous-manipulation/data/my_iris.yaml"
#iris_filename = FindDataResource("iiwa_shelve_and_bins_science_robotics.yaml")
iris_regions = dict()
q = []
print(iris_filename)

## Configure IRIS

Here is the [Documentation](https://drake.mit.edu/doxygen_cxx/structdrake_1_1geometry_1_1optimization_1_1_iris_options.html). for tuning the IRIS algorithm

In [4]:
iris_options = IrisOptions()
iris_options.iteration_limit = 10
# increase num_collision_infeasible_samples to improve the (probabilistic)
# certificate of having no collisions.
iris_options.num_collision_infeasible_samples = 3
iris_options.require_sample_point_is_contained = True
iris_options.relative_termination_threshold = 0.01
iris_options.termination_threshold = -1
iris_options.configuration_space_margin = 1e-3  # Example of a smaller margin

# Additional options for this notebook:

# If use_existing_regions_as_obstacles is True, then iris_regions will be
# shrunk by regions_as_obstacles_margin, and then passed to
# iris_options.configuration_obstacles.
use_existing_regions_as_obstacles = True
regions_as_obstacles_scale_factor = 0.95

# We can compute some regions in parallel.
num_parallel = mp.cpu_count()

In [7]:



def _CalcRegion(name, seed):
    builder = DiagramBuilder()
    plant = AddMultibodyPlantSceneGraph(builder, 0.0)[0]
    LoadRobot(plant)
    plant.Finalize()
    diagram = builder.Build()
    diagram_context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(diagram_context)
    plant.SetPositions(plant_context, seed)
    if use_existing_regions_as_obstacles:
        iris_options.configuration_obstacles = [
            ScaleHPolyhedron(r, regions_as_obstacles_scale_factor)
            for k, r in iris_regions.items()
            if k != name
        ]
        for h in iris_options.configuration_obstacles:
            _CheckNonEmpty(h)
    else:
        iris_options.configuration_obstacles = None
    display(f"Computing region for seed: {name}")
    start_time = time.time()
    hpoly = IrisInConfigurationSpace(plant, plant_context, iris_options)
    display(
        f"Finished seed {name}; Computation time: {(time.time() - start_time):.2f} seconds"
    )

    _CheckNonEmpty(hpoly)
    reduced = hpoly.ReduceInequalities()
    _CheckNonEmpty(reduced)

    return reduced


def GenerateRegion(name, seed):
    global iris_regions
    iris_regions[name] = _CalcRegion(name, seed)
    SaveIrisRegionsYamlFile(f"{iris_filename}.autosave", iris_regions)


def GenerateRegions(seed_dict, verbose=True):
    if use_existing_regions_as_obstacles:
        # Then run serially
        for k, v in seed_dict.items():
            GenerateRegion(k, v)
        return

    loop_time = time.time()
    with mp.Pool(processes=num_parallel) as pool:
        new_regions = pool.starmap(_CalcRegion, [[k, v] for k, v in seed_dict.items()])

    if verbose:
        print("Loop time:", time.time() - loop_time)

    global iris_regions
    iris_regions.update(dict(list(zip(seed_dict.keys(), new_regions))))


def DrawRobot(query_object: QueryObject, meshcat_prefix: str, draw_world: bool = True):
    rgba = Rgba(0.7, 0.7, 0.7, 0.3)
    role = Role.kProximity
    # This is a minimal replication of the work done in MeshcatVisualizer.
    inspector = query_object.inspector()
    for frame_id in inspector.GetAllFrameIds():
        if frame_id == inspector.world_frame_id():
            if not draw_world:
                continue
            frame_path = meshcat_prefix
        else:
            frame_path = f"{meshcat_prefix}/{inspector.GetName(frame_id)}"
        frame_path.replace("::", "/")
        frame_has_any_geometry = False
        for geom_id in inspector.GetGeometries(frame_id, role):
            path = f"{frame_path}/{geom_id.get_value()}"
            path.replace("::", "/")
            meshcat.SetObject(path, inspector.GetShape(geom_id), rgba)
            meshcat.SetTransform(path, inspector.GetPoseInFrame(geom_id))
            frame_has_any_geometry = True

        if frame_has_any_geometry:
            X_WF = query_object.GetPoseInWorld(frame_id)
            meshcat.SetTransform(frame_path, X_WF)


def VisualizeRegion(region_name, num_to_draw=30, draw_illustration_role_once=True):
    """
    A simple hit-and-run-style idea for visualizing the IRIS regions:
    1. Start at the center. Pick a random direction and run to the boundary.
    2. Pick a new random direction; project it onto the current boundary, and run along it. Repeat
    """

    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    LoadRobot(plant)
    plant.Finalize()
    if draw_illustration_role_once:
        MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyMutableContextFromRoot(context)
    scene_graph_context = scene_graph.GetMyContextFromRoot(context)

    global iris_regions
    region = iris_regions[region_name]

    q = region.ChebyshevCenter()
    plant.SetPositions(plant_context, q)
    diagram.ForcedPublish(context)

    query = scene_graph.get_query_output_port().Eval(scene_graph_context)
    DrawRobot(query, f"{region_name}/0", True)

    rng = np.random.default_rng()
    nq = plant.num_positions()
    prog = MathematicalProgram()
    qvar = prog.NewContinuousVariables(nq, "q")
    prog.AddLinearConstraint(region.A(), 0 * region.b() - np.inf, region.b(), qvar)
    cost = prog.AddLinearCost(np.ones((nq, 1)), qvar)

    for i in range(1, num_to_draw):
        direction = rng.standard_normal(nq)
        cost.evaluator().UpdateCoefficients(direction)

        result = Solve(prog)
        assert result.is_success()

        q = result.GetSolution(qvar)
        plant.SetPositions(plant_context, q)
        query = scene_graph.get_query_output_port().Eval(scene_graph_context)
        DrawRobot(query, f"{region_name}/{i}", False)


def VisualizeRegions():
    for k in iris_regions.keys():
        meshcat.Delete()
        VisualizeRegion(k)
        button_name = f"Visualizing {k}; Press for next region"
        meshcat.AddButton(button_name, "Enter")
        print("Press Enter to visualize the next region")
        while meshcat.GetButtonClicks(button_name) < 1:
            time.sleep(1.0)
        meshcat.DeleteButton(button_name)


# TODO(russt): See https://github.com/RobotLocomotion/drake/pull/19520
class PoseSelector(LeafSystem):
    def __init__(
        self,
        body_index=None,
    ):
        LeafSystem.__init__(self)
        self._body_index = body_index
        self.DeclareAbstractInputPort(
            "body_poses", AbstractValue.Make([RigidTransform()])
        )
        self.DeclareAbstractOutputPort(
            "pose",
            lambda: AbstractValue.Make(RigidTransform()),
            self.CalcOutput,
        )

    def CalcOutput(self, context, output):
        body_poses = self.get_input_port().Eval(context)
        output.set_value(body_poses[self._body_index])

## Defining Robot and Enviroment

These is where the enviroment (table, shelf, bins) and the robot (Kinova Gen 3 Lite) are imported and loaded

In [6]:
def LoadRobot(plant: MultibodyPlant) -> Body:
    """Setup your plant, and return the body corresponding to your
    end-effector."""
    parser = Parser(plant)
    
    robot_model = add_custom_robot(plant)[0]
    gripper_model = add_custom_wsg(plant)[0]
    table_model = add_table(plant)[0]
    shelf_model = add_shelf(plant)[0]
    bin_model = add_bin(plant)[0]
    #alphabet_soup_model, mustard_model, cherries_model  = add_objects(plant)
    weld_gripper_to_robot(plant, robot_model, gripper_model, ee_link_name="FINAL_JOINT_FRAME")

    


    gripper = plant.GetModelInstanceByName("final_gripper")
    end_effector_body = plant.GetBodyByName("body_gripper", gripper)
    return end_effector_body


# Sometimes it's useful to use inverse kinematics to find the seeds. You might
# need to adapt this to your robot. This helper takes an end-effector frame, E,
# and a desired pose for that frame in the world coordinates, X_WE.
def MyInverseKinematics(X_WE, plant=None, context=None):
    if not plant:
        plant = MultibodyPlant(0.0)
        LoadRobot(plant)
        plant.Finalize()
    if not context:
        context = plant.CreateDefaultContext()
    # E = ee_body.body_frame()
    gripper_model = plant.GetModelInstanceByName("final_gripper")
    frame = plant.GetFrameByName("tool_center_point", gripper_model)

    ik = InverseKinematics(plant, context)

    ik.AddPositionConstraint(
        frame, [0, 0, 0], plant.world_frame(), X_WE.translation(), X_WE.translation()
    )

    ik.AddOrientationConstraint(
        frame, RotationMatrix(), plant.world_frame(), X_WE.rotation(), 0.001
    )

    prog = ik.get_mutable_prog()
    q = ik.q()

    q0 = plant.GetPositions(context)
    prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
    prog.SetInitialGuess(q, q0)
    result = Solve(ik.prog())
    if not result.is_success():
        print("IK failed")
        return None
    plant.SetPositions(context, result.GetSolution(q))
    return result.GetSolution(q)

In [ ]:
# This is a handy way to test your model.
v = ModelVisualizer(meshcat=meshcat)
LoadRobot(v.parser().plant())
v.Run()

In [8]:
def initialize_dictionaries(alphabet_soup_model, mustard_model, cherries_model):
    object_dictionary = {
        alphabet_soup_model: {"frame_name": "body_alphabet_soup", "level": "Upper_level", "p_AQ_lower": [-0.01, -0.01, 0.035], "p_AQ_upper": [0.01,0.01,0.075]},
        mustard_model: {"frame_name": "body_mustard", "level": "Middle_level", "p_AQ_lower": [-0.035, -0.01, 0.02], "p_AQ_upper": [0.02,0.01,0.1]},
        cherries_model: {"frame_name": "body_cherries", "level": "Lower_level", "p_AQ_lower": [-0.035, -0.01, 0.02], "p_AQ_upper": [0.02,0.01,0.04]},
    }

    initial_guess_dictionary = {
        "Upper_level": np.array([0.08272460326162771, 0.14916581203514745, 1.5243876747212592, 0.2842089939295549, 0.1440074552134592, -0.2981349776251228], dtype=np.float64), 
        "Middle_level": np.array([0.18712170333911837, 0.0737186397351014, 1.8561300106972425, -0.23452860383219773, 0.21507161400030814, 0.26623220176260004], dtype=np.float64),
        "Lower_level": np.array([0.43617725045954725, -1.1199212481107446, 2.1184269163304155, -1.047214945101428, 1.5942907993927722, 1.5847549429295138], dtype=np.float64)
    }

    return object_dictionary, initial_guess_dictionary


def add_constraints(ik, plant, plant_context, gripper_frame, object_frame, object_pose, object_p_AQ_lower, object_p_AQ_upper):
    """Adds constraints to the IK problem."""
    # Fix object pose
    ik.AddPositionConstraint(
        frameB=object_frame,
        p_BQ=[0, 0, 0],
        frameA=plant.world_frame(),
        p_AQ_lower=object_pose.translation(),
        p_AQ_upper=object_pose.translation(),
    )
    ik.AddOrientationConstraint(
        frameAbar=object_frame,
        R_AbarA=RotationMatrix(),
        frameBbar=plant.world_frame(),
        R_BbarB=object_pose.rotation(),
        theta_bound=0.0,
    )

    # Gripper position constraints
    ik.AddPositionConstraint(
        frameB=gripper_frame,
        p_BQ=[0, 0.0, 0.0],
        frameA=object_frame,
        p_AQ_lower=object_p_AQ_lower,
        p_AQ_upper=object_p_AQ_upper,
    )

    # Desired orientation
    desired_rotation_matrix = RotationMatrix(RollPitchYaw([0, 1.5708, 0]))
    ik.AddOrientationCost(
        frameAbar=plant.world_frame(),
        R_AbarA=desired_rotation_matrix,
        frameBbar=gripper_frame,
        R_BbarB=RotationMatrix(),
        c=100.0,
    )

    ik.AddMinimumDistanceLowerBoundConstraint(0.001, 0.05)


def set_initial_guess(plant,plant_context, initial_guess_dictionary, location):
    """Sets the initial guess for the IK problem."""
    positions_name = plant.GetPositionNames()
    robot_position_indices = []
    robot_positions_names = ["KINOVA_GEN3_LITE_J0_q", "KINOVA_GEN3_LITE_J1_q", 
                         "KINOVA_GEN3_LITE_J2_q", "KINOVA_GEN3_LITE_J3_q", 
                         "KINOVA_GEN3_LITE_J4_q", "KINOVA_GEN3_LITE_J5_q"]

    for outer_iterator in range(6):
        iterator = 0
        for position_name in positions_name:
            if position_name == robot_positions_names[outer_iterator]:
                robot_position_indices.append(iterator)
                break
            iterator += 1

    q0 = plant.GetPositions(plant_context)
    q0[robot_position_indices] = initial_guess_dictionary[location]   
    return q0



def solve_ik(ik, q0):
    """Sets up and solves the IK problem."""
    prog = ik.get_mutable_prog()
    q = ik.q()
    prog.AddQuadraticErrorCost(10*np.identity(len(q)), q0, q)
    prog.SetInitialGuess(q, q0)
    result = Solve(prog)
    return result, q

def create_scene_and_multibody_plant():
    """
    Creates the scene and MultibodyPlant, adds models, and finalizes the plant.

    Returns:
        A tuple containing the builder, plant, scene graph, and model instances of the robot, gripper, table, and shelf.
    """
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    robot_model = add_custom_robot(plant)[0]
    gripper_model = add_custom_wsg(plant)[0]
    table_model = add_table(plant)[0]
    shelf_model = add_shelf(plant)[0]
    bin_model = add_bin(plant)[0]
    #alphabet_soup_model, mustard_model, cherries_model  = add_objects(plant)
    alphabet_soup_model, mustard_model, cherries_model  = None, None, None

    
    weld_gripper_to_robot(plant, robot_model, gripper_model, ee_link_name="FINAL_JOINT_FRAME")
    plant.Finalize()
    
    return builder, plant, scene_graph, robot_model, gripper_model, table_model, shelf_model, alphabet_soup_model, mustard_model, cherries_model

def setup_diagram_and_context(builder, plant, scene_graph):
    """
    Sets up the diagram and context for the simulation.

    Args:
        builder: The DiagramBuilder to which the plant and scene graph are added.
        plant: The MultibodyPlant containing the robot and gripper.
        scene_graph: The SceneGraph for visualization.

    Returns:
        A tuple containing the diagram, context, plant context, and initial positions.
    """
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder,
        scene_graph,
        meshcat,
        MeshcatVisualizerParams(delete_prefix_initialization_event=False),
    )

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    q0 = plant.GetPositions(plant_context)
    
    return diagram, context, plant_context, q0, visualizer



In [ ]:
# Create the scene/plant
builder, plant, scene_graph, robot_model, gripper_model, table_model, shelf_model, alphabet_soup_model, mustard_model, cherries_model = create_scene_and_multibody_plant()
diagram, context, plant_context, q0, visualizer = setup_diagram_and_context(
    builder, plant, scene_graph
)

# (Optional) Clear any previous MeshCat elements
meshcat.Delete()
meshcat.DeleteAddedControls()

# Initialize dictionaries
object_dictionary, initial_guess_dictionary = initialize_dictionaries(alphabet_soup_model, mustard_model, cherries_model)

# Randomly pick one model and its associated attributes
object_model, attributes = random.choice(list(object_dictionary.items()))    

# Get the frame of the object
object_frame = plant.GetFrameByName(attributes["frame_name"], object_model)
gripper_frame = plant.GetFrameByName("tool_center_point", gripper_model)    

# Build the IK problem
ik = InverseKinematics(plant, plant_context)

# We'll fix the object at its initial pose in the world:
object_pose = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName(attributes["frame_name"], object_model))

# Add constraints
add_constraints(ik, plant, plant_context, gripper_frame, object_frame, object_pose, attributes["p_AQ_lower"], attributes["p_AQ_upper"])

q0 = set_initial_guess(plant,plant_context, initial_guess_dictionary, attributes["level"])

result, q = solve_ik(ik, q0)

# 4) Solve!
if result.is_success():
    print("IK success!")
    q_sol = result.GetSolution(q)
    # Update the plant_context to visualize the solution
    plant.SetPositions(plant_context, q_sol)
else:
    print("IK failure.")     
diagram.ForcedPublish(context)

## Run IRIS on manually-specified seeds

In [ ]:
def get_default_position():
    plant = MultibodyPlant(0.0)
    LoadRobot(plant)
    plant.Finalize()
    context = plant.CreateDefaultContext()
    return plant.GetPositions(context)


# Note: The order of the seeds matters when we are using existing regions as
# configuration_obstacles.
seeds = OrderedDict()
seeds["Home Position"] = get_default_position()
seeds["Above Shelve"] = MyInverseKinematics(
    RigidTransform(RollPitchYaw(0, np.pi/2, 0), [0.25, 0, 0.5])
)
seeds["Top Rack"] = MyInverseKinematics(
    RigidTransform(RollPitchYaw(0, np.pi/2, 0), [0.25, 0, 0.5])
)
seeds["Middle Rack"] = MyInverseKinematics(
    RigidTransform(RollPitchYaw(0, np.pi/2, 0), [0.25, 0, 0.5])
)
seeds["Left Bin"] = MyInverseKinematics(
    RigidTransform(RollPitchYaw(0.0, np.pi/2,0), [0.0, 0.3, 0.3])
)
seeds["Right Bin"] = MyInverseKinematics(
    RigidTransform(RollPitchYaw(0.0, np.pi/2,0), [0.0, -0.0, 0.5])
)
#seeds["Front to Shelve"] = np.array([0, 0.2, 0, -2.09, 0, -0.3, np.pi / 2])
seeds["Left to Shelve"] = np.array([0.8, 0.7, 0, -1.6, 0, 0, np.pi / 2])
seeds["Right to Shelve"] = np.array([-0.8, 0.7, 0, -1.6, 0, 0, np.pi / 2])

In [ ]:
def create_seeds(plant, object_dictionary, initial_guess_dictionary, diagram, plant_context):
    """
    Creates seeds for the inverse kinematics problem using provided logic.
    
    Args:
        builder: DiagramBuilder instance.
        plant: MultibodyPlant instance.
        scene_graph: SceneGraph instance.
        object_dictionary: Dictionary of object attributes.
        initial_guess_dictionary: Dictionary of initial guess values.
    
    Returns:
        A list of seeds, each representing a solution for an object in the scene.
    """
    seeds = []
    
    for object_model, attributes in object_dictionary.items():
        # Get the frame of the object
        object_frame = plant.GetFrameByName(attributes["frame_name"], object_model)
        gripper_frame = plant.GetFrameByName("tool_center_point", gripper_model)
        
        # Build the IK problem
        ik = InverseKinematics(plant, plant_context)
        
        # Fix the object pose in the world
        object_pose = plant.EvalBodyPoseInWorld(
            plant_context, plant.GetBodyByName(attributes["frame_name"], object_model)
        )
        
        # Add constraints
        add_constraints(
            ik,
            plant,
            plant_context,
            gripper_frame,
            object_frame,
            object_pose,
            attributes["p_AQ_lower"],
            attributes["p_AQ_upper"],
        )
        
        # Set the initial guess
        q0 = set_initial_guess(plant, plant_context, initial_guess_dictionary, attributes["level"])
        
        # Solve the IK problem
        result, q = solve_ik(ik, q0)
        if result.is_success():
            print(f"IK success for {attributes['frame_name']}!")
            q_sol = result.GetSolution(q)
            seeds.append(q_sol)
        else:
            print(f"IK failure for {attributes['frame_name']}!")
    
    return seeds


# Create the scene and plant
(builder, plant, scene_graph, robot_model, 
gripper_model, table_model, shelf_model, 
alphabet_soup_model, mustard_model, cherries_model) = create_scene_and_multibody_plant()

# Setup the diagram and context
diagram, context, plant_context, q0, visualizer = setup_diagram_and_context( builder, plant, scene_graph, )


# Initialize dictionaries
object_dictionary, initial_guess_dictionary = initialize_dictionaries(alphabet_soup_model, mustard_model, cherries_model)

# Generate seeds for the IK problem
seeds = create_seeds(plant, object_dictionary, initial_guess_dictionary, diagram, plant_context)

# Optionally, visualize the solutions for the generated seeds
for i, seed in enumerate(seeds):
    print(f"Seed {i + 1}: {seed}")
    plant.SetPositions(plant_context, seed)
    diagram.ForcedPublish(context)
    time.sleep(1)


In [ ]:
from pydrake.math import RollPitchYaw
from pydrake.math import RigidTransform
# (Optional) Clear any previous MeshCat elements
meshcat.Delete()
meshcat.DeleteAddedControls()

def make_rigid_transform(rpy, xyz):
    """
    Create a RigidTransform from roll-pitch-yaw (in radians) and (x, y, z).
    
    Args:
        rpy: [roll, pitch, yaw] in radians
        xyz: [x, y, z] in meters

    Returns:
        A drake RigidTransform instance.
    """
    roll, pitch, yaw = rpy
    x, y, z = xyz
    rotation = RollPitchYaw(roll, pitch, yaw).ToRotationMatrix()
    translation = [x, y, z]
    return RigidTransform(rotation, translation)

from pydrake.all import (
    InverseKinematics,
    RotationMatrix,
    RigidTransform,
    Solve,
)

def create_seeds_for_transforms(
    plant,
    plant_context,
    diagram,
    gripper_model,
    transform_list,
    initial_guess_dictionary=None,
    constraints_dict=None
):
    """
    Solve IK for a list of desired end-effector transforms.

    Args:
        plant: A MultibodyPlant instance.
        plant_context: The plant's Context.
        diagram: The Diagram containing plant (optional if you need it for referencing).
        gripper_model: ModelInstanceIndex or similar reference for the gripper.
        transform_list: A list of dictionaries, each containing { "rpy": [...], "xyz": [...], ... }.
        initial_guess_dictionary: (Optional) For setting robot initial joint guesses per transform.
        constraints_dict: (Optional) Dictionary of constraints or bounding boxes for orientation, position, etc.

    Returns:
        A list of joint solutions (each is a numpy array) corresponding to each transform.
    """
    seeds = []

    # Access the end-effector (tool) frame.
    gripper_frame = plant.GetFrameByName("tool_center_point", gripper_model)

    for i, transform_spec in enumerate(transform_list):
        print(f"\n--- Solving IK for Transform #{i+1} ---")

        # Build the IK problem
        ik = InverseKinematics(plant, plant_context)

        # Convert [roll, pitch, yaw, x, y, z] into a RigidTransform
        desired_pose = make_rigid_transform(
            rpy=transform_spec["rpy"],
            xyz=transform_spec["xyz"],
        )

        # Add position constraint. 
        #   Example: The origin of the gripper_frame must be within ± some tolerance of desired_pose’s translation.
        pos_lower = desired_pose.translation() - 1e-2
        pos_upper = desired_pose.translation() + 1e-2
        ik.AddPositionConstraint(
            frameB=gripper_frame, 
            p_BQ=[0, 0, 0],           # The point Q in the gripper frame, e.g., the "tip"
            frameA=plant.world_frame(), 
            p_AQ_lower=pos_lower,
            p_AQ_upper=pos_upper
        )

        # Add orientation constraint.
        #   Example: The gripper frame’s orientation must be within a small angular tolerance from the desired R.
        desired_R = desired_pose.rotation()
        ik.AddOrientationConstraint( 
            frameAbar=plant.world_frame(),
            R_AbarA=desired_R,
            frameBbar=gripper_frame,
            R_BbarB=RotationMatrix(),
            theta_bound=0.1  # rad tolerance
        )
        ik.AddMinimumDistanceLowerBoundConstraint(0.0001, 0.001)

        # If you want to add custom bounding boxes for position or orientation from constraints_dict, do so here
        if constraints_dict is not None:
            # e.g. p_AQ_lower, p_AQ_upper, etc.
            pass

        # Set an initial guess for the solver
        if initial_guess_dictionary:
            q0 = set_initial_guess_from_dictionary(
                plant, plant_context, initial_guess_dictionary, transform_spec
            )
        else:
            # Or simply use the current context as the guess
            q0 = plant.GetPositions(plant_context).copy()

        ik.prog().SetInitialGuess(ik.q(), q0)

        # Solve the IK
        result = Solve(ik.prog())
        if result.is_success():
            print(f"IK succeeded for transform #{i+1}.")
            q_sol = result.GetSolution(ik.q())
            seeds.append(q_sol)
        else:
            print(f"IK failed for transform #{i+1}.")
            # You may want to append None or skip
            seeds.append(None)

    return seeds

def set_initial_guess_from_dictionary(plant, plant_context, initial_guess_dictionary, transform_spec):
    """
    A placeholder function that picks an initial guess (q0) for the solver
    based on your dictionary plus the transform's index or type.
    Modify as needed for your usage.
    """
    # For demonstration, just return the current positions:
    return plant.GetPositions(plant_context).copy()

import time

# Build the scene and plant as usual
(builder, plant, scene_graph, robot_model, 
 gripper_model, table_model, shelf_model, 
 alphabet_soup_model, mustard_model, cherries_model) = create_scene_and_multibody_plant()

# Setup the diagram and context
diagram, context, plant_context, q0, visualizer = setup_diagram_and_context(builder, plant, scene_graph)

# Example transforms list: each item has RPY (radians) and xyz (meters)
# Suppose we want to place the end-effector at these two poses.
transform_list = [
    {
        "rpy": [0, np.pi/2, 0],      # roll, pitch, yaw
        "xyz": [0.35, 0.0, 0.6] # x, y, z
    },
    {
        "rpy": [0, np.pi/2, 0],
        "xyz": [0.35, 0, 0.35]
    },
    {
        "rpy": [0, np.pi/2, 0],
        "xyz": [0.25, 0.5, 0.15]
    }
]

"""
        [0.35, 0, 0.535],        
        [0.35, 0, 0.28],
        [0.35, 0, 0.01]"""
# Optionally define your own initial guesses or constraints
initial_guess_dictionary = {}
constraints_dict = {
    # Example custom bounding:
    # "p_AQ_lower": np.array([x_min, y_min, z_min]),
    # "p_AQ_upper": np.array([x_max, y_max, z_max]),
}
 
# Now create seeds for each transform
ik_solutions = create_seeds_for_transforms(
    plant,
    plant_context,
    diagram,
    gripper_model,
    transform_list,
    initial_guess_dictionary,
    constraints_dict,
)

# Visualize each solution
for i, seed in enumerate(ik_solutions):
    if seed is None:
        print(f"Solution {i+1}: None (IK Failed)")
        continue
    print(f"Solution {i+1}: {seed}")
    plant.SetPositions(plant_context, seed)
    diagram.ForcedPublish(context)
    time.sleep(1)



In [ ]:
# v = ModelVisualizer(meshcat=meshcat)
# LoadRobot(v.parser().plant())
# v.Run(position=seeds["Top Rack"], loop_once=True)
seeds = ik_solutions
print(seeds)

In [ ]:
from pydrake.multibody.tree import JointIndex

for i in range(plant.num_joints()):
    joint = plant.get_joint(JointIndex(i))
    print(joint.name(), joint.type_name())


In [ ]:
# You generate one region at a time, e.g.
# GenerateRegion("Top Rack", seeds["Top Rack"])

# Or (re-)generate all of the regions
# This will run in parallel iff use_existing_regions_as_obstacles is False
seeds = {str(i): seed for i, seed in enumerate(seeds)}
print(seeds)
# Keep only the first two elements of seeds
#seeds = {k: seeds[k] for k in list(seeds)[:2]}

if running_as_notebook:
    iris_regions = dict()  # reset the iris regions
    GenerateRegions(seeds)

In [ ]:
# VisualizeRegion("Top Rack")
VisualizeRegions()

## End-effector Teleop

In [ ]:
from collections import namedtuple
from copy import copy

from pydrake.all import Context, Meshcat, MeshcatPoseSliders, SceneGraph

RpyXyz = namedtuple("RpyXyz", ("roll", "pitch", "yaw", "x", "y", "z"))


# Creates an indicator "light" (a colored sphere in the workspace)
# TODO(russt): use meshcat.SetProperty(path, "color", rgba) instead
def IrisIndicator(
    scene_graph: SceneGraph,
    scene_graph_context: Context,
    meshcat: Meshcat,
    iris_regions: Dict[str, HPolyhedron],
    p_WIndicator: list,
):
    radius = 0.1

    meshcat.SetTransform("iris_indicator", RigidTransform(p_WIndicator))

    query_object = scene_graph.get_query_output_port().Eval(scene_graph_context)
    has_collisions = query_object.HasCollisions()
    if has_collisions:
        meshcat.SetObject("iris_indicator", Sphere(radius), Rgba(1, 0, 0, 1))

    in_any_region = False
    for r in iris_regions.values():
        if r.PointInSet(q):
            if has_collisions:
                print("You found a counter-example!")
                # TODO(russt): Automatically shrink the iris region.
            elif not in_any_region:
                in_any_region = True
                meshcat.SetObject("iris_indicator", Sphere(radius), Rgba(0, 1, 0, 1))

    if not has_collisions and not in_any_region:
        meshcat.SetObject("iris_indicator", Sphere(radius), Rgba(0.5, 0.5, 0.5, 1))


def EndEffectorTeleop():
    meshcat.Delete()
    meshcat.DeleteAddedControls()

    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 0.0)
    ee_body = LoadRobot(plant)
    plant.Finalize()

    # Set up teleop widgets.
    sliders = builder.AddSystem(
        MeshcatPoseSliders(
            meshcat,
            lower_limit=RpyXyz(roll=0, pitch=-0.5, yaw=-np.pi, x=-0.6, y=-0.8, z=0.0),
            upper_limit=RpyXyz(
                roll=2 * np.pi, pitch=np.pi, yaw=np.pi, x=0.8, y=0.3, z=1.1
            ),
        )
    )
    print(
        """
Control the end-effector position/orientation with the sliders or the keyboard:
  ┌───────┬┬───────┬┬───────┐      ┌───────┬┬───────┬┬───────┐
  │   Q   ││   W   ││   E   │      │   U   ││   I   ││   O   │
  │ roll- ││ pitch+││ roll+ │      │   z-  ││   y+  ││   z+  │
  ├───────┼┼───────┼┼───────┤      ├───────┼┼───────┼┼───────┤
  ├───────┼┼───────┼┼───────┤      ├───────┼┼───────┼┼───────┤
  │   A   ││   S   ││   D   │      │   J   ││   K   ││   L   │
  │  yaw- ││ pitch-││  yaw+ │      │   x-  ││   y-  ││   x+  │
  └───────┴┴───────┴┴───────┘      └───────┴┴───────┴┴───────┘
"""
    )

    AddDefaultVisualization(builder, meshcat)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyMutableContextFromRoot(context)
    scene_graph_context = scene_graph.GetMyContextFromRoot(context)
    sliders_context = sliders.GetMyContextFromRoot(context)

    global q
    if len(q) == 0:
        q = plant.GetPositions(plant_context)
    plant.SetPositions(plant_context, q)
    X_WE = plant.EvalBodyPoseInWorld(plant_context, ee_body)
    sliders.SetPose(X_WE)

    iris_button_name = "Compute new IRIS region"
    meshcat.AddButton(iris_button_name)
    iris_button_clicks = 0

    stop_button_name = "Stop End Effector Teleop"
    print(
        f"Press the '{stop_button_name}' button in Meshcat to continue or press 'Escape'"
    )
    meshcat.AddButton(stop_button_name, "Escape")

    diagram.ForcedPublish(context)
    if not running_as_notebook:
        return

    while meshcat.GetButtonClicks(stop_button_name) < 1:
        # Check if the sliders have changed.
        new_X_WE = copy(sliders.get_output_port().Eval(sliders_context))

        if meshcat.GetButtonClicks(iris_button_name) > iris_button_clicks:
            iris_button_clicks = meshcat.GetButtonClicks(iris_button_name)
            # TODO(russt): Get the name from meshcat (#19666)
            region_name = "EETeleopRegion"
            region_num = 0
            while f"{region_name}{region_num}" in iris_regions.keys():
                region_num += 1
            meshcat.AddButton("Generating region (please wait)")
            GenerateRegion(
                f"{region_name}{region_num}", plant.GetPositions(plant_context)
            )
            meshcat.DeleteButton("Generating region (please wait)")
        elif X_WE.IsExactlyEqualTo(new_X_WE):
            time.sleep(1e-3)
            continue

        X_WE = new_X_WE
        q_ik = MyInverseKinematics(X_WE, plant, plant_context)
        if q_ik is not None:
            q = q_ik
            plant.SetPositions(plant_context, q)
            IrisIndicator(
                scene_graph,
                scene_graph_context,
                meshcat,
                iris_regions,
                p_WIndicator=[1, 1, 1],
            )
            diagram.ForcedPublish(context)

    meshcat.DeleteAddedControls()
    q = plant.GetPositions(plant_context)


EndEffectorTeleop()

### Sometimes it's useful to reset the teleop to the default (home) position

In [ ]:
q = get_default_position()

## Joint Teleop

In [ ]:
def JointTeleop():
    meshcat.Delete()
    meshcat.DeleteAddedControls()
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    LoadRobot(plant)
    plant.Finalize()

    sliders = builder.AddSystem(JointSliders(meshcat, plant))

    AddDefaultVisualization(builder, meshcat)
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyMutableContextFromRoot(context)
    scene_graph_context = scene_graph.GetMyContextFromRoot(context)
    sliders_context = sliders.GetMyContextFromRoot(context)

    global q
    if len(q) == 0:
        q = plant.GetPositions(plant_context)
    sliders.SetPositions(q)

    # Implements a version of JointSliders.run() which can also watch for extra UI events.
    iris_button_name = "Compute new IRIS region"
    meshcat.AddButton(iris_button_name)
    iris_button_clicks = 0

    stop_button_name = "Stop Joint Teleop"
    print(
        f"Press the '{stop_button_name}' button in Meshcat to continue or press 'Escape'"
    )
    meshcat.AddButton(stop_button_name, "Escape")

    diagram.ForcedPublish(context)
    if not running_as_notebook:
        return

    while meshcat.GetButtonClicks(stop_button_name) < 1:
        # Check if the sliders have changed.
        old_positions = plant.GetPositions(plant_context)
        new_positions = sliders.get_output_port().Eval(sliders_context)

        if meshcat.GetButtonClicks(iris_button_name) > iris_button_clicks:
            iris_button_clicks = meshcat.GetButtonClicks(iris_button_name)
            # TODO(russt): Get the name from meshcat (#19666)
            region_name = "JointTeleopRegion"
            region_num = 0
            while f"{region_name}{region_num}" in iris_regions.keys():
                region_num += 1
            meshcat.AddButton("Generating region (please wait)")
            GenerateRegion(f"{region_name}{region_num}", new_positions)
            meshcat.DeleteButton("Generating region (please wait)")
        elif np.array_equal(new_positions, old_positions):
            time.sleep(1e-3)
            continue

        # Publish the new positions.
        plant.SetPositions(plant_context, new_positions)
        IrisIndicator(
            scene_graph,
            scene_graph_context,
            meshcat,
            iris_regions,
            p_WIndicator=[1, 1, 1],
        )
        diagram.ForcedPublish(context)

    meshcat.DeleteAddedControls()


JointTeleop()

## Random sampling

Simple rejection sampling can work, but we have some much nicer methods coming soon.

# Introspection

## Visualize the current IRIS region connectivity

For planning trajectories, it's important to have some non-empty intersection between the convex sets. Here is a simple utility the visualization the connectivity as a graph.

In [ ]:
def VisualizeConnectivity():
    graph = pydot.Dot("IRIS region connectivity")
    keys = list(iris_regions.keys())
    for k in keys:
        graph.add_node(pydot.Node(k))
    for i in range(len(keys)):
        v1 = iris_regions[keys[i]]
        for j in range(i + 1, len(keys)):
            v2 = iris_regions[keys[j]]
            if v1.IntersectsWith(v2):
                graph.add_edge(pydot.Edge(keys[i], keys[j], dir="both"))
    display(SVG(graph.create_svg()))



VisualizeConnectivity()

## Visualize a region

In [ ]:
if running_as_notebook and iris_regions:
    VisualizeRegions()

## Plan trajectories with GCS Trajectory Optimization

In [20]:
from pydrake.geometry.optimization import GraphOfConvexSetsOptions, Point
from pydrake.planning import GcsTrajectoryOptimization


def PublishPositionTrajectory(
    trajectory, root_context, plant, visualizer, time_step=1.0 / 33.0
):
    """
    Args:
        trajectory: A Trajectory instance.
    """
    
    print("running animation")
    plant_context = plant.GetMyContextFromRoot(root_context)
    visualizer_context = visualizer.GetMyContextFromRoot(root_context)

    visualizer.StartRecording(False)

    for t in np.append(
        np.arange(trajectory.start_time(), trajectory.end_time(), time_step),
        trajectory.end_time(),
    ):
        root_context.SetTime(t)
        plant.SetPositions(plant_context, trajectory.value(t))
        visualizer.ForcedPublish(visualizer_context)

    visualizer.StopRecording()
    visualizer.PublishRecording()


def GcsTrajOpt(q_start, q_goal):
    if not iris_regions:
        print(
            "No IRIS regions loaded. Make some IRIS regions then come back and try this again."
        )
        return
    assert len(q_start) == len(q_goal)
    assert len(q_start) == iris_regions[next(iter(iris_regions))].ambient_dimension()

    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    LoadRobot(plant)
    plant.Finalize()
    AddDefaultVisualization(builder, meshcat)
    diagram = builder.Build()

    gcs = GcsTrajectoryOptimization(len(q_start))
    # TODO(russt): AddRegions should take named regions.
    regions = gcs.AddRegions(list(iris_regions.values()), order=1)
    source = gcs.AddRegions([Point(q_start)], order=0)
    target = gcs.AddRegions([Point(q_goal)], order=0)
    gcs.AddEdges(source, regions)
    gcs.AddEdges(regions, target)
    gcs.AddTimeCost()
    gcs.AddVelocityBounds(
        plant.GetVelocityLowerLimits(), plant.GetVelocityUpperLimits()
    )

    options = GraphOfConvexSetsOptions()
    options.preprocessing = True
    options.max_rounded_paths = 5
    start_time = time.time()
    traj, result = gcs.SolvePath(source, target, options)
    print(f"GCS solved in {time.time() - start_time} seconds")
    if not result.is_success():
        print("Could not find a feasible path from q_start to q_goal")
        return

    PublishPositionTrajectory(
        traj,
        diagram.CreateDefaultContext(),
        plant,
        diagram.GetSubsystemByName("meshcat_visualizer(illustration)"),
    )



In [ ]:
# clear the meshcat visualizer
meshcat.Delete()
meshcat.DeleteAddedControls()


assert (
    seeds
), "The examples here use the 'manually-specified seeds' from the section above. Please run that section first, or populate your own start and end configurations."

GcsTrajOpt(seeds["0"], seeds["2"])

In [ ]:
lower = plant.GetVelocityLowerLimits()
upper = plant.GetVelocityUpperLimits()

# Replace -inf with a large negative and inf with a large positive
large = 1e6
lower = np.where(np.isinf(lower), -large, lower)
upper = np.where(np.isinf(upper),  large, upper)

print("Velocity Lower Limits:", lower)
print("Velocity Upper Limits:", upper)

In [ ]:
for k in range(4 if running_as_notebook else 0):
    GcsTrajOpt(seeds["Top Rack"], seeds["Left Bin"])
    GcsTrajOpt(seeds["Left Bin"], seeds["Middle Rack"])
    GcsTrajOpt(seeds["Middle Rack"], seeds["Right Bin"])
    GcsTrajOpt(seeds["Right Bin"], seeds["Top Rack"])

# Save your regions back to disk

Note that by default, this notebook will autosave a backup after making each region.

In [ ]:
if running_as_notebook:
    SaveIrisRegionsYamlFile(iris_filename, iris_regions)

# Clean up

For good measure, let's delete any temporary license files now.

In [ ]:
if os.path.exists("/tmp/mosek.lic"):
    os.remove("/tmp/mosek.lic")